# Lecture 9: pandas, merge, linear regression

IF you're running this on Google Colab, and only then, should you run this cell:

In [ ]:
# !! Run this on Google Colab only.
from google.colab import drive
drive.mount('/content/drive')
DATA_FILEPATH = "/content/drive/My Drive/risk_free_rate.ft"

Download data from Yahoo Finance:

In [ ]:
# Use this version of Yahoo Finance, because later versions
# changed syntax to multiple indices.
!pip install yfinance==0.2.44
!pip install fredapi
import os
import pathlib

import numpy as np
import pandas as pd
import fredapi
import yfinance
import matplotlib.pyplot as plt

RISK_FREE = "risk_free"
MARKET_RATE = "market_rate"
FED_FUNDS = "DFF"
SP500 = "^SPX"

START_DATE = "1986-01-02"
TODAY = "2024-11-07"

def download_yfinance(ticker):

    # Download financial data from Yahoo Finance. No need for API key.
    data = yfinance.download(ticker, start=START_DATE, end=TODAY)[["Close"]]

    # Use this redundant syntax to avoid a warning from Pandas.
    data = data.rename(columns={"Close": ticker})

    return data

Import risk-free rate data, saved in "feather" format (or use an API key to download from the Federal Reserve Economic Database):

In [ ]:
def get_risk_free_and_market_rates():

    if os.path.exists(DATA_FILEPATH):
        return pd.read_feather(DATA_FILEPATH)

    with open("api_key.txt") as f:
        api_key = f.read().strip()

    fred = fredapi.Fred(api_key)
    risk_free = fred.get_series(FED_FUNDS)
    risk_free = np.exp(np.log(1 + risk_free.div(100)) / 250) - 1
    risk_free.name = RISK_FREE

    market = download_yfinance(SP500)
    market[SP500] = market[SP500].pct_change()
    market = market.rename(columns={SP500: MARKET_RATE})

    merged = pd.merge(market,
                      risk_free,
                      how="inner",
                      left_index=True,
                      right_index=True)

    merged.to_feather(DATA_FILEPATH)

    return merged

get_risk_free_and_market_rates()

Download a stock and compute its percentage change.

In [ ]:
def download_and_compute_growth(ticker):
    stock = download_yfinance(ticker)
    growth = stock.pct_change()
    print(growth.tail())
    return growth

Download Microsoft, Apple, merge them, and drop missing values (e.g. if a company went public before the other).

In [ ]:
apple = download_and_compute_growth("AAPL")
microsoft = download_and_compute_growth("MSFT")

# Index of the series = date.
merged = pd.merge(apple,
                  microsoft,
                  how="inner",
                  left_index=True,
                  right_index=True)

plt.hist(microsoft)
plt.hist(apple)
merged.dropna(inplace=True)
merged.describe()

Run a regression of one on the other.

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

def regression(merged):
  x = merged["AAPL"].values.reshape(-1, 1)
  y = merged["MSFT"].values.reshape(-1, 1)
  reg = LinearRegression().fit(x, y)

  # Hack to show the two regressions in the same range.
  x_axis = np.arange(-0.6, 0.4, 0.1).reshape(-1, 1)
  y_predicted = reg.predict(x_axis)

  plt.scatter(x, y)
  plt.plot(x_axis, y_predicted, c="r")
  print("Coefficient: ", reg.coef_[0, 0])

regression(merged)

Let's drop the observation at the left to see the effect of outliers on linear regression: the regression line is slightly steeper when we drop it. The coefficient is about 3% higher, due to a single observation!

In [ ]:
from sklearn.linear_model import LinearRegression

clean = merged.loc[merged["AAPL"] >= -0.4]

regression(clean)

# CAPM

Time-allowing: run a Capital Asset-Pricing Model regression with two stocks of your choice.